In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import matplotlib.pyplot as plt
import seaborn as sns
from training_models import Autoencoder, DenoisingAutoencoder, AutoencoderEnsemble
from monitor_models import Autocovariance, AugmentationCrossCovariances, RepresentationCrossCovariances
from os.path import join
import pandas as pd
from directory import RESULTS_DIR, PLOTS_DIR

2024-03-25 14:32:30.898134: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 14:32:32.841555: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 14:32:38.470310: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64:/usr/local/cuda/lib64:/home/onur/anaconda3/envs/tf/lib/
2024-03-25 14:32:38

In [2]:
EXP_NAME = "exp_test"
EVAL_FILE_NAME = "summary.csv"
LATEX_FILE_NAME = "summary.tex"

In [3]:
def _to_sigma_notation(row):
    return f"{row['ROC-AUC']} +- {row['ROC-AUC-STD']}"

def _create_method_name(row):
    training_model_name = row["Training model"]
    monitoring_model_name = row["Monitoring model"]

    if training_model_name == Autoencoder().name and monitoring_model_name == Autocovariance().name:
        return "Autocovariance"
    
    if training_model_name == DenoisingAutoencoder().name and monitoring_model_name == Autocovariance().name:
        return "Autocovariance \n(Denoising)"
    
    if training_model_name == Autoencoder().name and monitoring_model_name == AugmentationCrossCovariances().name:
        return "Augmentation Cross-covariances"
    
    if training_model_name == DenoisingAutoencoder().name and monitoring_model_name == AugmentationCrossCovariances().name:
        return "Augmentation Cross-covariances \n(Denoising)"
    
    if training_model_name == AutoencoderEnsemble().name and monitoring_model_name == RepresentationCrossCovariances().name:
        return "Representation Cross-covariances"
    
def _get_min_validation_loss_epochs(df_summary):
    df_summary_grouped = df_summary.groupby(['Method', 'Training dataset', 'Testing dataset', 'Split'])
    df_summary_min_validation = df_summary.loc[df_summary_grouped['Validation loss'].idxmin()]
    df_summary_min_validation.reset_index()
    return df_summary_min_validation

def _add_rocauc_sigma_notation(df_summary):
    _df_summary = df_summary.copy()
    _df_summary_grouped = _df_summary.groupby(['Method', 'Training dataset', 'Testing dataset'])
    
    _df_summary_mean= _df_summary_grouped['ROC-AUC'].mean().reset_index()
    _df_summary_std = _df_summary_grouped['ROC-AUC'].std().reset_index()
    
    _df_summary_mean["ROC-AUC-STD"] = _df_summary_std['ROC-AUC']
    _df_summary_mean = _df_summary_mean.round(3)
    _df_summary_mean["ROC-AUC"] = _df_summary_mean.apply(_to_sigma_notation, axis=1)
    return _df_summary_mean

In [4]:
df_summary = pd.read_csv(join(RESULTS_DIR, EXP_NAME, EVAL_FILE_NAME))
df_summary["Method"] = df_summary.apply(_create_method_name, axis=1)
df_summary.drop(["Training model", "Monitoring model"], axis=1, inplace=True)

2024-03-25 14:32:46.418930: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-25 14:32:46.418964: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: nova98
2024-03-25 14:32:46.418967: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: nova98
2024-03-25 14:32:46.419053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 525.85.12
2024-03-25 14:32:46.419116: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 525.85.12
2024-03-25 14:32:46.419118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 525.85.12
2024-03-25 14:32:46.433211: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI D

In [5]:
df_summary = _get_min_validation_loss_epochs(df_summary)
df_summary = _add_rocauc_sigma_notation(df_summary)
df_summary.sort_values(by=['Training dataset', 'Testing dataset', 'Method'], inplace=True)
df_summary.to_latex(join(RESULTS_DIR, EXP_NAME, LATEX_FILE_NAME))


/tmp/ipykernel_37477/2683659800.py:4: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df_summary.to_latex(join(RESULTS_DIR, EXP_NAME, LATEX_FILE_NAME))
